In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from __future__ import print_function
import argparse
import os

import numpy as np
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from keras.layers import merge
from keras.optimizers import Adam, SGD, RMSprop
from keras.preprocessing.image import list_pictures, array_to_img

from image_ext import list_pictures_in_multidir, load_imgs_asarray

np.random.seed(2016)

/home/nakazawa_atsushi/anaconda3/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/nakazawa_atsushi/anaconda3/envs/py3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def create_fcn00(input_size):
    inputs = Input((3, input_size[1], input_size[0]))

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv5)
    pool5 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv5)

    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool5)
    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv6)

    up7 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv6), conv5])
    conv7 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(up7)
    conv7 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv7)

    up8 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv7), conv4])
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(up8)
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv8)

    up9 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv8), conv3])
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(up9)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv9)

    up10 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv9), conv2])
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(up10)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv10)

    up11 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv10), conv1])
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(up11)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv11)

    conv12 = Conv2D(1, (1, 1), activation='sigmoid', padding='same', data_format='channels_first')(conv11)
    #conv12 = Conv2D(1, 1, 1)(conv11)
    
    fcn = Model(inputs=inputs, outputs=conv12)

    return fcn

In [3]:
def create_fcn01(input_size):
    inputs = Input((3, input_size[1], input_size[0]))

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv4)

    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool4)
    conv6 = Conv2D(512, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv6)
    
    up8 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv6), conv4])
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(up8)
    conv8 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv8)

    up9 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv8), conv3])
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(up9)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv9)

    up10 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv9), conv2])
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(up10)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv10)

    up11 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv10), conv1])
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(up11)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv11)

    conv12 = Conv2D(1, (1, 1), activation='sigmoid', padding='same', data_format='channels_first')(conv11)
    #conv12 = Conv2D(1, 1, 1)(conv11)
    
    fcn = Model(inputs=inputs, outputs=conv12)

    return fcn

In [4]:
def create_fcn02(input_size):
    inputs = Input((3, input_size[1], input_size[0]))

    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2), data_format='channels_first')(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv4)

    up9 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv4), conv3])
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(up9)
    conv9 = Conv2D(128, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv9)

    up10 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv9), conv2])
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(up10)
    conv10 = Conv2D(64, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv10)

    up11 = Concatenate(axis=1)([UpSampling2D(size=(2, 2), data_format='channels_first')(conv10), conv1])
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(up11)
    conv11 = Conv2D(32, (3, 3), activation='relu', padding='same', data_format='channels_first')(conv11)

    conv12 = Conv2D(1, (1, 1), activation='sigmoid', padding='same', data_format='channels_first')(conv11)
    #conv12 = Conv2D(1, 1, 1)(conv11)
    
    fcn = Model(inputs=inputs, outputs=conv12)

    return fcn

In [5]:
def dice_coef(y_true, y_pred):
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred)
    return (2.*intersection + 1) / (K.sum(y_true) + K.sum(y_pred) + 1)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [6]:
def load_fnames(paths):
    f = open(paths)
    data1 = f.read()
    f.close()
    lines = data1.split('\n')
    #print(len(lines))
    # 最終行は空行なので消す
    del(lines[len(lines)-1])
    #print(len(lines))
    return lines

In [7]:
def make_fnames(fnames,fpath,fpath_mask,mask_ext):
    fnames_img = [];
    fnames_mask= [];
    
    for i in range(len(fnames)):
        fnames_img.append(fpath + '/' + fnames[i]);
        fnames_mask.append(fpath_mask + '/' + mask_ext + fnames[i]);
        
    return [fnames_img,fnames_mask]

In [8]:
if __name__ == '__main__':
    
    target_size = (224, 224)
    dpath_this = './'
    dname_checkpoints = 'checkpoints01'
    dname_outputs = 'outputs'
    fname_architecture = 'architecture.json'
    fname_weights = "model_weights_{epoch:02d}.h5"
    fname_stats = 'stats01.npz'
    dim_ordering = 'channels_first'
    
    # Read Learning Data
    fnames = load_fnames('data/list_train_01.txt')
    [fpaths_xs_train,fpaths_ys_train] = make_fnames(fnames,'data/img','data/mask','OperatorA_')
    
    X_train = load_imgs_asarray(fpaths_xs_train, grayscale=False, target_size=target_size,
                                dim_ordering=dim_ordering)
    Y_train = load_imgs_asarray(fpaths_ys_train, grayscale=True, target_size=target_size,
                                dim_ordering=dim_ordering) 
    
    # Read Validation Data
    fnames = load_fnames('data/list_valid_01.txt')
    [fpaths_xs_valid,fpaths_ys_valid] = make_fnames(fnames,'data/img','data/mask','OperatorA_')
    
    X_valid = load_imgs_asarray(fpaths_xs_valid, grayscale=False, target_size=target_size,
                                dim_ordering=dim_ordering)
    Y_valid = load_imgs_asarray(fpaths_ys_valid, grayscale=True, target_size=target_size,
                                dim_ordering=dim_ordering)     

In [9]:
    print('==> ' + str(len(X_train)) + ' training images loaded')
    print('==> ' + str(len(Y_train)) + ' training masks loaded')
    print('==> ' + str(len(X_valid)) + ' validation images loaded')
    print('==> ' + str(len(Y_valid)) + ' validation masks loaded')

==> 1452 training images loaded
==> 1452 training masks loaded
==> 527 validation images loaded
==> 527 validation masks loaded


In [10]:
    # 前処理
    print('computing mean and standard deviation...')
    mean = np.mean(X_train, axis=(0, 2, 3))
    std = np.std(X_train, axis=(0, 2, 3))
    print('==> mean: ' + str(mean))
    print('==> std : ' + str(std))

    print('saving mean and standard deviation to ' + fname_stats + '...')
    stats = {'mean': mean, 'std': std}
    np.savez(fname_stats, **stats)
    print('==> done')

    print('globally normalizing data...')
    for i in range(3):
        X_train[:, i] = (X_train[:, i] - mean[i]) / std[i]
        X_valid[:, i] = (X_valid[:, i] - mean[i]) / std[i]
    Y_train /= 255
    Y_valid /= 255
    print('==> done')

computing mean and standard deviation...
==> mean: [130.65465  91.2685   76.63643]
==> std : [55.2817   43.990963 43.113483]
saving mean and standard deviation to stats01.npz...
==> done
globally normalizing data...
==> done


In [11]:
    # モデルを作成
    print('creating model...')
    model = create_fcn02(target_size)
    model.summary()

creating model...
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 224, 224)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 224, 224) 896         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 224, 224) 9248        conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 32, 112, 112) 0           conv2d_2[0][0]                   
___________________________________________________________________________________________

In [12]:
    # 損失関数，最適化手法を定義
    adam = Adam(lr=1e-5)
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.1, nesterov=True)
    #rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    model.compile(optimizer=adam, loss=dice_coef_loss, metrics=[dice_coef])
    
    # 構造・重みを保存するディレクトリーの有無を確認
    dpath_checkpoints = os.path.join(dpath_this, dname_checkpoints)
    if not os.path.isdir(dpath_checkpoints):
        os.mkdir(dpath_checkpoints)
    # モデルの構造を保存
    json_string = model.to_json()
    fpath_architecture = os.path.join(dpath_checkpoints, fname_architecture)
    with open(fpath_architecture, "w", encoding="utf8") as f:
        f.write(json_string)
    # 重みを保存するためのオブジェクトを用意
    fpath_weights = os.path.join(dpath_checkpoints, fname_weights)
    checkpointer = ModelCheckpoint(filepath=fpath_weights, save_best_only=False)        
        

In [13]:
    # トレーニングを開始
    print('start training...')
    history = model.fit(X_train, Y_train, batch_size=32, epochs=300, verbose=1,
                  shuffle=True, validation_data=(X_valid, Y_valid), callbacks=[checkpointer])

start training...
Train on 1452 samples, validate on 527 samples
Epoch 1/300
1452/1452 [==============================] - 67s 46ms/step - loss: -0.0168 - dice_coef: 0.0168 - val_loss: -0.0215 - val_dice_coef: 0.0215
Epoch 2/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.0215 - dice_coef: 0.0215 - val_loss: -0.0306 - val_dice_coef: 0.0306
Epoch 3/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.0525 - dice_coef: 0.0525 - val_loss: -0.1105 - val_dice_coef: 0.1105
Epoch 4/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.1839 - dice_coef: 0.1839 - val_loss: -0.2014 - val_dice_coef: 0.2014
Epoch 5/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.2937 - dice_coef: 0.2937 - val_loss: -0.3496 - val_dice_coef: 0.3496
Epoch 6/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.4098 - dice_coef: 0.4098 - val_loss: -0.4281 - val_dice_coef: 0.4281
Epoch 7/300
1452/1452 [======

1452/1452 [==============================] - 50s 34ms/step - loss: -0.8059 - dice_coef: 0.8059 - val_loss: -0.7904 - val_dice_coef: 0.7904
Epoch 55/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8011 - dice_coef: 0.8011 - val_loss: -0.8121 - val_dice_coef: 0.8121
Epoch 56/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8129 - dice_coef: 0.8129 - val_loss: -0.8159 - val_dice_coef: 0.8159
Epoch 57/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8137 - dice_coef: 0.8137 - val_loss: -0.8067 - val_dice_coef: 0.8067
Epoch 58/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8149 - dice_coef: 0.8149 - val_loss: -0.8119 - val_dice_coef: 0.8119
Epoch 59/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8194 - dice_coef: 0.8194 - val_loss: -0.8202 - val_dice_coef: 0.8202
Epoch 60/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8166 - dice_coef: 

Epoch 108/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8475 - dice_coef: 0.8475 - val_loss: -0.8447 - val_dice_coef: 0.8447
Epoch 109/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8401 - dice_coef: 0.8401 - val_loss: -0.8453 - val_dice_coef: 0.8453
Epoch 110/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8297 - dice_coef: 0.8297 - val_loss: -0.8411 - val_dice_coef: 0.8411
Epoch 111/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8459 - dice_coef: 0.8459 - val_loss: -0.8403 - val_dice_coef: 0.8403
Epoch 112/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8474 - dice_coef: 0.8474 - val_loss: -0.8186 - val_dice_coef: 0.8186
Epoch 113/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8446 - dice_coef: 0.8446 - val_loss: -0.8432 - val_dice_coef: 0.8432
Epoch 114/300
1452/1452 [==============================] - 50s 34ms/step - loss: -

1452/1452 [==============================] - 50s 34ms/step - loss: -0.8562 - dice_coef: 0.8562 - val_loss: -0.8286 - val_dice_coef: 0.8286
Epoch 162/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8642 - dice_coef: 0.8642 - val_loss: -0.8546 - val_dice_coef: 0.8546
Epoch 163/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8670 - dice_coef: 0.8670 - val_loss: -0.8343 - val_dice_coef: 0.8343
Epoch 164/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8606 - dice_coef: 0.8606 - val_loss: -0.8585 - val_dice_coef: 0.8585
Epoch 165/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8671 - dice_coef: 0.8671 - val_loss: -0.8594 - val_dice_coef: 0.8594
Epoch 166/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8636 - dice_coef: 0.8636 - val_loss: -0.8587 - val_dice_coef: 0.8587
Epoch 167/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8657 - dice_

1452/1452 [==============================] - 50s 34ms/step - loss: -0.8746 - dice_coef: 0.8746 - val_loss: -0.8612 - val_dice_coef: 0.8612
Epoch 215/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8793 - dice_coef: 0.8793 - val_loss: -0.8529 - val_dice_coef: 0.8529
Epoch 216/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8727 - dice_coef: 0.8727 - val_loss: -0.8657 - val_dice_coef: 0.8657
Epoch 217/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8675 - dice_coef: 0.8675 - val_loss: -0.8620 - val_dice_coef: 0.8620
Epoch 218/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8790 - dice_coef: 0.8790 - val_loss: -0.8641 - val_dice_coef: 0.8641
Epoch 219/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8788 - dice_coef: 0.8788 - val_loss: -0.8435 - val_dice_coef: 0.8435
Epoch 220/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8745 - dice_

1452/1452 [==============================] - 50s 34ms/step - loss: -0.8868 - dice_coef: 0.8868 - val_loss: -0.8352 - val_dice_coef: 0.8352
Epoch 268/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8837 - dice_coef: 0.8837 - val_loss: -0.8598 - val_dice_coef: 0.8598
Epoch 269/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8861 - dice_coef: 0.8861 - val_loss: -0.8612 - val_dice_coef: 0.8612
Epoch 270/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8841 - dice_coef: 0.8841 - val_loss: -0.8565 - val_dice_coef: 0.8565
Epoch 271/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8866 - dice_coef: 0.8866 - val_loss: -0.8687 - val_dice_coef: 0.8687
Epoch 272/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8784 - dice_coef: 0.8784 - val_loss: -0.8535 - val_dice_coef: 0.8535
Epoch 273/300
1452/1452 [==============================] - 50s 34ms/step - loss: -0.8848 - dice_

In [18]:
    print(history)

In [21]:
    import pickle
    
    f = open('history.pkl','w')
    pickle.dump(history.history,f)
    f.close

TypeError: can't pickle _thread.lock objects